# Model search

## Getting started

This script will go through assembling the main components of a complete pipeline for counting seals in high-resolution satellite imagery. 

<img src="jupyter_notebook_images/pipeline.png">
** *environmental covariates not currently incorporated **

If you followed the *training_set_generation* jupyter notebook (also present in this repo), you should have training sets generated and hyperparameter sets to try out, and be ready to search for a best performing model setup. Models will be evaluated at the three levels of the detection pipeline: **1)** their ability to identify scenes with seals, **2)** retrieve seal haul outs **3)** and count individual seals. Before training and validating model/hyperparameter combinations, we need to load the required python modules. Running this script will also display a list of training classes. 


In [2]:
# import required packages
import os
import rasterio
import pandas as pd
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt
import matplotlib as mpl
from functools import reduce
from utils.model_library import * 

%matplotlib inline
mpl.rcParams['figure.dpi']= 400


# display class names
class_names = sorted([subdir for subdir in os.listdir('./training_sets/training_set_vanilla/training')])
print(class_names)

['crabeater', 'crack', 'emperor', 'glacier', 'ice-sheet', 'marching-emperor', 'open-water', 'other', 'pack-ice', 'rock', 'weddell']


## Visualizing training images (Optional)

To get a better sense for what the training set is like, the next cell will display a few random images from the training classes. Displayed images are extracted from a pool of ~70000 training images. 

In [ ]:
# store images
images = []

# loop over labels
for label in class_names:
    for path, _, files in os.walk('./training_sets/training_set_vanilla/training/{}'.format(label)):
        files = np.random.choice(files, 5)
        for filename in files:
            images.append(np.asarray(Image.open(os.path.join(path, filename))))

images = np.array(images)

# display images 
ncols=len(class_names)
nindex, height, width, intensity = images.shape
nrows = nindex//ncols
assert nindex == nrows*ncols
result = (images.reshape(nrows, ncols, height, width, intensity)
          .swapaxes(1,2)
          .reshape(height*nrows, width*ncols, intensity))

plt.imshow(result)
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)
plt.show()
    


## Training - haulout detector

The first step to find a best performing model is to train different model setups using our training set. To keep track of which combinations we have tried and the specifics of each model setup, we will create a pandas DataFrame with model definitions.

In [3]:
# generate model combinations
combinations_haul = {'model_architecture': ['Resnet18'] * 4 + ['NasnetA'] * 4,
                     'training_dir': ['training_set_vanilla', 'training_set_multiscale_A'] * 4,
                     'hyperparameter_set': ['A'] * 4 + ['B'] * 4,
                     'cv_weights': ['NO', 'NO', 'A', 'A'] * 2,
                     'output_name': ['model{}'.format(i) for i in range(1,9)]}

combinations_haul = pd.DataFrame(combinations_haul)

# create folders for resulting files
for mdl in combinations_haul['output_name']:
    if not os.path.exists("./saved_models/haulout/{}".format(mdl)):
        os.makedirs("./saved_models/haulout/{}".format(mdl)) 


We can then provide model combinations created above as arguments to the training script, *train_sealnet.py*. A list of required arguments can be displayed by running the cell below.

In [ ]:
%run train_sealnet.py -h

In [ ]:
# iterate over combinations
for row in combinations_haul.iterrows():
    t_dir, arch, hyp_st, cv_wgt, out = row[1]['training_dir'], row[1]['model_architecture'], row[1]['hyperparameter_set'], row[1]['cv_weights'], row[1]['output_name']
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./saved_models/haulout/{}/'.format(out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch --hyperparameter_set=$hyp_st --cv_weights=$cv_wgt --output_name=$out
    
      

## Training - single seal detector



Similar to training a model at the haulout level, we start by training a number of model combinations. To keep track of which combinations we have tried and the specifics of each model setup, we will create a pandas DataFrame with model definitions.

In [16]:
# generate model combinations
combinations_single = {'model_architecture': ['WideResnetA'] * 4,
                       'training_dir': ['training_set_vanilla', 'training_set_multiscale_A'] * 2,
                       'hyperparameter_set': ['C'] * 4,
                       'cv_weights': ['NO', 'NO', 'A', 'A'],
                       'output_name': ['model{}'.format(i) for i in range(1,5)]}

combinations_single = pd.DataFrame(combinations_single)

# create folders for resulting files
for mdl in combinations_single['output_name']:
    if not os.path.exists("./saved_models/single_seal/{}".format(mdl)):
        os.makedirs("./saved_models/single_seal/{}".format(mdl)) 

Similar to the haulout models, we can provide model combinations created above as arguments to the training script, *train_sealnet.py*. 

In [17]:
# iterate over combinations
for row in combinations_single.iterrows():
    t_dir, arch, hyp_st, cv_wgt, out = row[1]['training_dir'], row[1]['model_architecture'], row[1]['hyperparameter_set'], row[1]['cv_weights'], row[1]['output_name']
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./saved_models/single_seal/{}/'.format(out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch --hyperparameter_set=$hyp_st --cv_weights=$cv_wgt --output_name=$out
    


training model1

Epoch 1/30
----------
training Loss: 0.0616 Acc: 0.5316
validation Loss: 0.3756 Acc: 0.6643
training time: 0.0h 1m 23s

Epoch 2/30
----------
training Loss: 0.0499 Acc: 0.6131
validation Loss: 0.3693 Acc: 0.6852
training time: 0.0h 2m 46s

Epoch 3/30
----------
training Loss: 0.0463 Acc: 0.6395
validation Loss: 0.3615 Acc: 0.6668
training time: 0.0h 4m 9s

Epoch 4/30
----------
training Loss: 0.0448 Acc: 0.6495
validation Loss: 0.3048 Acc: 0.7343
training time: 0.0h 5m 33s

Epoch 5/30
----------
training Loss: 0.0433 Acc: 0.6628
validation Loss: 0.3437 Acc: 0.6872
training time: 0.0h 6m 57s

Epoch 6/30
----------
training Loss: 0.0416 Acc: 0.6740
validation Loss: 0.3373 Acc: 0.6710
training time: 0.0h 8m 20s

Epoch 7/30
----------
training Loss: 0.0411 Acc: 0.6803
validation Loss: 0.3202 Acc: 0.6830
training time: 0.0h 9m 43s

Epoch 8/30
----------
training Loss: 0.0401 Acc: 0.6873
validation Loss: 0.3305 Acc: 0.6888
training time: 0.0h 11m 8s

Epoch 9/30
----------
t

training Loss: 0.0308 Acc: 0.6618
validation Loss: 0.3233 Acc: 0.7151
training time: 0.0h 15m 14s

Epoch 9/30
----------
training Loss: 0.0301 Acc: 0.6694
validation Loss: 0.3263 Acc: 0.7263
training time: 0.0h 16m 37s

Epoch 10/30
----------
training Loss: 0.0294 Acc: 0.6741
validation Loss: 0.3228 Acc: 0.7014
training time: 0.0h 18m 3s

Epoch 11/30
----------
training Loss: 0.0291 Acc: 0.6782
validation Loss: 0.2781 Acc: 0.7544
training time: 0.0h 19m 28s

Epoch 12/30
----------
training Loss: 0.0288 Acc: 0.6818
validation Loss: 0.2853 Acc: 0.7460
training time: 0.0h 20m 52s

Epoch 13/30
----------
training Loss: 0.0278 Acc: 0.6896
validation Loss: 0.2957 Acc: 0.7442
training time: 0.0h 22m 16s

Epoch 14/30
----------
training Loss: 0.0277 Acc: 0.6911
validation Loss: 0.3169 Acc: 0.6914
training time: 0.0h 23m 39s

Epoch 15/30
----------
training Loss: 0.0273 Acc: 0.6934
validation Loss: 0.2971 Acc: 0.7174
training time: 0.0h 25m 3s

Epoch 16/30
----------
training Loss: 0.0274 Acc: 


Epoch 1/30
----------
Traceback (most recent call last):
  File "train_sealnet.py", line 245, in <module>
    main()
  File "train_sealnet.py", line 241, in main
    num_epochs=hyperparameters[args.hyperparameter_set]['epochs'])
  File "train_sealnet.py", line 158, in train_model
    outputs = model(inputs)
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/nn/modules/module.py", line 491, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/bento/PycharmProjects/Seals_branches/Master/Seals/custom_architectures/wide_resnet.py", line 92, in forward
    out = F.avg_pool2d(out, 7)
RuntimeError: Given input size: (64x4x4). Calculated output size: (64x0x0). Output size is too small at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/THCUNN/generic/SpatialAveragePooling.cu:63

training model8

Epoch 1/30
----------
Traceback (most recent call last):
  File "train_sealnet.py", line 245, in <module>
    main()
  File "train_sealnet.py", line 241, in main


## Validation - haulout level

We can now load the models we just trained to get measurements of precision and recall for all positive classes. For every model combination we trained, *validate_sealnet.py* will run a full validation round and write given label/correct label pairs to a .csv file. The resulting .csv file is then imported by an R script, *plot_confusion_matrix.R*, which saves a confusion matrix figure and a .csv spreadsheet with precision and recall for all classes of interest. 

In [21]:
# DataFrame to combine all metrics 
comb_prec_recall = pd.DataFrame()

# iterate over trained models
for row in combinations_haul.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st, out = row[1]['training_dir'], row[1]['model_architecture'], row[1]['hyperparameter_set'], row[1]['output_name']
    
    # check if model file is available
    if "{}.tar".format(out) not in os.listdir('./saved_models/haulout/{}/'.format(out)): 
        print('{} has not been trained yet'.format(out))
        continue
        
    else:
        print()
        !echo validating $out
        print()
        
        # run validation
        !python validate_sealnet_haulout.py --training_dir=$t_dir --model_architecture=$arch --hyperparameter_set=$hyp_st --model_name=$out
        
        # extract performance
        !Rscript plot_confusion_matrix.R $out
        
        # accumulate performance scores
        comb_prec_recall = comb_prec_recall.append(pd.read_csv('./saved_models/haulout/{}/{}_haul_prec_recall.csv'.format(out, out)))
    
    
# Write combined metrics to csv and plot combined metrics
comb_prec_recall.to_csv('./saved_models/haulout/pooled_haul_prec_recall.csv')
!Rscript plot_comparison.R './saved_models/haulout/pooled_haul_prec_recall.csv' './saved_models/haulout/haul_comparison_plot.png'
    


validating model1

Validation complete in 0.0h 0m 27s
Validation Acc: 0.898638
Loading required package: methods

validating model2

Validation complete in 0.0h 1m 1s
Validation Acc: 0.863181
Loading required package: methods

validating model3

Validation complete in 0.0h 0m 27s
Validation Acc: 0.863309
Loading required package: methods

validating model4

Validation complete in 0.0h 0m 27s
Validation Acc: 0.731757
Loading required package: methods

validating model5

Validation complete in 0.0h 4m 50s
Validation Acc: 0.921891
Loading required package: methods

validating model6

Validation complete in 0.0h 4m 43s
Validation Acc: 0.917652
Loading required package: methods

validating model7

Validation complete in 0.0h 4m 44s
Validation Acc: 0.894142
Loading required package: methods

validating model8

Validation complete in 0.0h 4m 46s
Validation Acc: 0.859841
Loading required package: methods
Loading required package: methods
null device 
          1 


## Validation - scene level

The first step to validate models at the scene level is to tile out rasters to model input sizes. The following cell will search for all rasters in a dir, check if they are present in the scene_bank (see  *training_set_generation.ipynb*), store an affine matrix for that scene to go from the tile's index in the scene to projected coordinates and create tiles with the correct dimensions(including multi-scale models) for all model architectures in the 'combinations' DataFrame. 

In [ ]:
# load scene bank
scene_bank = pd.read_csv('./training_sets/seals_scene_bank.csv')

# point to raster dir
raster_dir = '/home/bento/imagery'

# create output folder
%mkdir './tiled_images'

# store data transforms
affine_transforms = {}

# loop through rasters creating tiled versions
print('\nTiling rasters:\n')
for path, _, files in os.walk(raster_dir):
    # filter rasters to include only ones present in the scene bank
    files = [file for file in files if file in pd.unique(scene_bank['scene'])]
    for count, filename in enumerate(files):
        filename_lower = filename.lower()
        input_path = (os.path.join(path, filename))
        # extract data transform to go from raster index to ESPG3031 coordinates
        with rasterio.open(input_path) as src:
            affine_transforms[filename] = [src.transform[1], src.transform[2], src.transform[0], src.transform[4], src.transform[5], src.transform[3]]
        # loop over models
        for row in combinations_haul.iterrows():
            # get model input size
            input_size = model_archs[row[1]['model_architecture']]
            ts_scales = training_sets[row[1]['training_dir']]['scale_bands']
            scales = [str(int(input_size * scale / ts_scales[0])) for scale in ts_scales]
            scale_bands = reduce(lambda x,y: x + '_' + y, scales)
            output_folder = './tiled_images/{}/{}'.format(input_path, scale_bands)
            # create a tiled out image for that model if it doesn't exist
            if os.path.exists(output_folder):
                print('  {} was already split into tiles'.format(filename))
                continue
            !python tile_raster.py --scale_bands=$scale_bands  --input_image=$input_path --output_folder='./tiled_images'
        print('\nProcessed {} out of {} rasters'.format(count + 1, len(files)))

# write data transforms to csv
affine_transforms = pd.DataFrame(affine_transforms)
affine_transforms.to_csv('affine_transforms.csv')

With rasters tiled to the correct dimensions we can now run validation at the scene level. This procedure will work as follows: for each model, classify all images tiled in the previous step and save classification results to a pandas DataFrame. Results will be compared with the scene_bank to measure precision and recall at detecting which scenes contain positive entries (i.e. seals, penguins or even both, depending on the scene_bank being used). Locations marked as having seal haulouts will be used to detect individual seals and validate models at the seal level. To get validation results, run the following cell: 

In [20]:
# loop through models classifying tiles
print('\nClassifying tiles:\n')
for row in combinations_haul.iterrows():
    # store model classifications to get seal locations for seal level validation
    val_scene_model = pd.DataFrame()
    
    # get model settings
    arch = row[1]['model_architecture']
    model_name = row[1]['output_name']
    t_dir = row[1]['training_dir']
    hyp_st = row[1]['hyperparameter_set']
    
    # check if model was already processed
    if "{}_scene_val.csv".format(model_name) in os.listdir('./saved_models/haulout/{}/'.format(model_name)): 
        print('  Tiles were already processed with {}'.format(model_name))
        continue
    
    # get scale bands
    input_size = model_archs[arch]['input_size']
    ts_scales = training_sets[t_dir]['scale_bands']
    scales = [str(int(input_size * scale / ts_scales[0])) for scale in ts_scales]
    scale_bands = reduce(lambda x,y: x + '_' + y, scales)
    
    # loop through tiled rasters
    scenes = os.listdir('./tiled_images')
    print('\nClassifying tiled scenes with {}:'.format(model_name))
    for count, scene in enumerate(scenes):
        input_folder = './tiled_images/{}/{}'.format(scene, scale_bands)
        %run predict_sealnet.py --training_dir=$t_dir --model_architecture=$arch --hyperparameter_set=$hyp_st --model_name=$model_name --data_dir=$input_folder --affine_transforms='affine_transforms.csv' --scene=$scene        
        val_scene_model = val_scene_model.append(pd.read_csv('./saved_models/haulout/{}/{}_scene_val_tmp.csv'.format(model_name, model_name)), ignore_index=True)
        print('\n  Classified {} out of {} tiled scenes'.format(count + 1, len(scenes)))
    
    # save final classifications to csv and cleanup temporary stats
    val_scene_model.to_csv('./saved_models/haulout/{}/{}_scene_val.csv'.format(model_name, model_name))
    to_rm = './saved_models/haulout/{}/{}_scene_val_tmp.csv'.format(model_name, model_name)
    !rm $to_rm

# get precision and recall for classification at all scene banks
comb_prec_rec = pd.DataFrame()

print('\nValidating models:\n')
# loop thorugh models again to get precision and recall
for count, row in enumerate(combinations_haul.iterrows()):
    # get model name
    model_name = row[1]['output_name']
    # get validation stats
    %run validate_sealnet_scene.py --model_name=$model_name
    comb_prec_rec = comb_prec_rec.append(pd.read_csv('./saved_models/haulout/{}/{}_scene_prec_recall.csv'.format(model_name, model_name)), ignore_index=True)
    print('  Validated {} out of {} models'.format(count + 1, len(combinations_haul)))
    
# save combined precision recall to csv and plot it
comb_prec_rec.to_csv('./saved_models/haulout/pooled_scene_prec_recall.csv', index=False)
!Rscript plot_comparison.R './saved_models/haulout/pooled_scene_prec_recall.csv' './saved_models/haulout/scene_comparison_plot.png'




Classifying tiles:

  Tiles were already processed with model1
  Tiles were already processed with model2
  Tiles were already processed with model3
  Tiles were already processed with model4
  Tiles were already processed with model5
  Tiles were already processed with model6
  Tiles were already processed with model7
  Tiles were already processed with model8

Validating models:

  Validated 1 out of 8 models
  Validated 2 out of 8 models
  Validated 3 out of 8 models
  Validated 4 out of 8 models
  Validated 5 out of 8 models
  Validated 6 out of 8 models
  Validated 7 out of 8 models
  Validated 8 out of 8 models
Loading required package: methods
null device 
          1 


## Validation -- single seal level

With seal haulout locations determined, we can go ahead and try to count seals inside flagged seal haul out. 

In [1]:
%run train_sealnet_count.py --training_dir="training_set_vanilla_count" --model_architecture='NasnetACount' --hyperparameter_set='B' --cv_weights='NO' --output_name='count-ception'


Epoch 1/5
----------
training Loss: 2.1029
validation Loss: 9.4041
training time: 0.0h 37m 56s

Epoch 2/5
----------
training Loss: 2.1184
validation Loss: 9.4700
training time: 1.0h 15m 25s

Epoch 3/5
----------
training Loss: 2.0883
validation Loss: 9.3499
training time: 1.0h 53m 40s

Epoch 4/5
----------
training Loss: 2.0652
validation Loss: 9.5114
training time: 2.0h 33m 18s

Epoch 5/5
----------


Process Process-21:
Process Process-23:
Process Process-24:
Process Process-22:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/bento/PycharmProjects/Seals_branches/Master/Seals/utils/data_loader_train_det.py", line 110, in __getitem__
    sample = self.loader(path)
  File "/home/bento/PycharmProjects/Seals_branches/Master/Seals/utils/data_loader_train_det.py", line 167, in default_loader
    return pil_loader(path)
  File "/home/bento/PycharmProjects/Seals_branches/Master/

In [1]:
%run train_sealnet_count_e2e.py --training_dir="training_set_vanilla_count" --model_architecture='NasnetAe2e' --hyperparameter_set='E' --cv_weights='NO' --output_name='NasnetAe2e_V'
%run train_sealnet_count_e2e.py --training_dir="training_set_multiscale_A_count" --model_architecture='NasnetAe2e' --hyperparameter_set='E' --cv_weights='NO' --output_name='NasnetAe2e_MS'

Epoch 1/10
----------
training Loss: 2.2607 Acc: 0.7001
validation Loss: 14.8348 Acc: 0.6339
training time: 1.0h 54m 43s

Epoch 2/10
----------
training Loss: 2.2659 Acc: 0.8595
validation Loss: 18.3803 Acc: 0.5907
training time: 3.0h 49m 29s

Epoch 3/10
----------
training Loss: 2.1439 Acc: 0.8928
validation Loss: 22.7197 Acc: 0.5870
training time: 5.0h 44m 12s

Epoch 4/10
----------
training Loss: 2.2004 Acc: 0.9147
validation Loss: 22.0916 Acc: 0.5890
training time: 7.0h 38m 49s

Epoch 5/10
----------
training Loss: 2.2068 Acc: 0.9235
validation Loss: 20.3537 Acc: 0.5889
training time: 9.0h 33m 22s

Epoch 6/10
----------
training Loss: 2.1304 Acc: 0.9363
validation Loss: 22.2404 Acc: 0.5895
training time: 11.0h 27m 54s

Epoch 7/10
----------
training Loss: 2.1302 Acc: 0.9412
validation Loss: 21.8812 Acc: 0.5890
training time: 13.0h 22m 26s

Epoch 8/10
----------
training Loss: 2.0879 Acc: 0.9478
validation Loss: 25.9171 Acc: 0.5904
training time: 15.0h 17m 2s

Epoch 9/10
----------


FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/single_seal/NasnetAe2e_V/NasnetAe2e_V.tar'

Epoch 1/10
----------


Process Process-54:
Process Process-51:
Process Process-53:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-52:
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/bento/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/bento/PycharmProjects/Seals_branches/Master/Seals/utils/data_loader_train_det.py", line 118, in __getitem__
    sample = self.loader(path)
  File "/home/bento/PycharmProjects/Seals_branches/Master/Seals/utils/data_loader_train_det.py", line 167, in default_loader
    return pil_loader(path)
  File "/home/bento/PycharmProjects/Seals_branches/Master/

In [2]:
%mkdir classified_images
for x in class_names:
    dir = './classified_images/' + x
    %mkdir -v $dir
    
%run predict_sealnet.py "training_set_vanilla" "NasnetA" "model5" "./to_classify"

NameError: name 'class_names' is not defined